In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import importlib

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from datasets.hdf5 import get_train_loaders
from unet3d.config import load_config
from unet3d.losses import get_loss_criterion
from unet3d.metrics import get_evaluation_metric
from unet3d.model import get_model
from unet3d.trainer import UNet3DTrainer
from unet3d.utils import get_logger
from unet3d.utils import get_number_of_learnable_parameters

import argparse

import os
import yaml
import numpy as np
import h5py
import matplotlib.pyplot as plt

# load data to see how it is structured

In [28]:
with h5py.File("resources/random_label3D.h5", 'r') as f:
    raw = f["raw"][:]
    label = f["label"][:]    


print('raw (input) data')
print('\t', 'unique values:', np.unique(raw))
print('\t', 'shape:', raw.shape)
print('\t', 'shape:', raw.dtype)

print()
print('label data')
print('\t', 'unique values:', np.unique(label))
print('\t', 'shape:', label.shape)
print('\t', 'shape:', label.dtype)


raw (input) data
	 unique values: [8.07995910e-08 5.25213452e-07 5.89760546e-07 ... 9.99999048e-01
 9.99999198e-01 9.99999818e-01]
	 shape: (128, 128, 128)
	 shape: float64

label data
	 unique values: [0 1]
	 shape: (128, 128, 128)
	 shape: int64


In [39]:
with h5py.File("h5_fractals/2.h5", 'r') as f:
    raw = f["raw"][:]
    label = f["label"][:]    


print('raw (input) data')
print('\t', 'unique values:', np.unique(raw))
print('\t', 'shape:', raw.shape)
print('\t', 'dtype:', raw.dtype)
print(np.sum(raw==1))

print('label data')
print('\t', 'unique values:', np.unique(label))
print('\t', 'shape:', label.shape)
print('\t', 'dtype:', label.dtype)
print(np.sum(label==1))

raw (input) data
	 unique values: [0. 1.]
	 shape: (128, 128, 128)
	 dtype: float64
3398
label data
	 unique values: [0 1]
	 shape: (128, 128, 128)
	 dtype: int64
6


# load model for predction

In [3]:
config_file = 'resources/train_config_ce.yaml'
config = yaml.load(open(config_file, 'r'))
model = get_model(config)
loaded_model = torch.load('3dunet/last_checkpoint.pytorch')
model.load_state_dict(loaded_model['model_state_dict'])

/home/gheimber/software/anaconda3/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


### load training set

In [4]:
with h5py.File("trachea_test/0.h5", 'r') as f:
    raw = f["raw"][:]
    label = f["label"][:]    
    

In [8]:
inp = torch.tensor(raw)
inp = raw[:32,:64,:64]
inp.shape

(32, 64, 64)

In [9]:
inp_2 = np.expand_dims(inp, 0)
inp_3 = np.expand_dims(inp_2, 0)
inp_3.shape

(1, 1, 32, 64, 64)

In [24]:
#prediction = model(torch.tensor(inp_3, dtype=torch.double))
prediction = model(torch.ones(1,1,128,128,128))
prediction_np = prediction.detach().numpy()

In [29]:
import ipyvolume as ipv

In [32]:
ipv.quickvolshow(np.squeeze(prediction_np[:,1,:,:,:]))